# Searching for and Plotting Lightcurves of Eclipsing Binary Stars

***

## Learning Goals
>Write three to five learning goals. A learning goal should describe what a reader should know or be able to do by the >end of the tutorial that they didn't know or couldn't do before.

By the end of this tutorial, you will:

- Use VizieR to load a digital catalog or table from a publication in a refereed astronomy journal. 
- Use `astroquery` to load the APOGEE Binary Radial Velocity catalog and narrow down a list of stellar binaries by a set of parameters. 
- Determine if a stellar system in the APOGEE catalog has lightcurve in the TESS Data for Asteroseismology Lightcurves archive from TASOC (the TESS Asterosesmic Science Operations Center).
- If so, plot the lightcurve using `lightkurve`.
- (Maybe?) Given a known eclipsing cadence, predict future eclipsing event dates/times for follow up observations.
- (Not this tutorial, likely a second one, but putting the idea here for future refrence) Cross-reference two catalogs to find objects that appear in both. 

## Introduction
>Write a short introduction explaining the purpose of the tutorial. Define any terms or common acronyms that your audience may not know. If you're using some kind of domain-specific astronomical symbol or unusual mathematical concept, make sure you define it (for example, in its mathematical form) and link to any definitions (from literature, Wikipedia, etc.).

>If there are background materials or resources that may be useful to the reader to provide additional context, you may link to it here. If your tutorial is a continuation from another tutorial, or there are other tutorials that would be useful for the reader to read before or after your tutorial, mention that here as well.

Just like extrasolar planets, binary star systems can be discovered by multiple methods. Three of the methods include:

1. Astrometry (explain, visual binary)
2. Radial velocity (explain, spectroscopic binary)
3. Eclipses. 

Notes: 
- Define eclipse, transit, binary, multiple. Asteroseismology! 
- The observations/missions used
- The Apache Point Observatory Galactic Evolution Experiment, APOGEE-1 and APOGEE-2 surveys... https://www.sdss.org/surveys/apogee-2/ 
- Something about, do binary system candidates identified with RV also have TESS lightcurves that show eclipses. What info can be gathered from each type of observation and why eclipse would add more information. Narrow down to e.g. short orbital periods, why most likely to find it with that. Choose ones that do it. 

In this tutorial, we will load a catalog of binary stellar systems discovered through the radial velocity method and investigate if those systems have also have lightcurves from the TESS mission that observed eclipsing.  


## Imports
>Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.

- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

JRK update when done.

In [1]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from astroquery.mast import Mast
from astroquery.mast import Observations
# Above were from the template, JRK review which ones end up applicable.
from astroquery.vizier import Vizier

***

## Loading the APOGEE Binary Radial Velocity catalog from VizieR

The catalogs and data tables produced for astrophysical publications in refereed journals are made accessible in digital form through the CDS VizieR catalog service. First, we'll use astroquery to find the tables associated with [Price-Whelan et al. 2018 (AJ, 156, 1, 18)](https://ui.adsabs.harvard.edu/abs/2018AJ....156...18P/abstract), a catalog of ~5000 binary compansions of evolved stars in APOGEE DR14. 

>Loading data and file information should appear within your main content, at the same time the data is going to be used, if possible. These elements of your tutorial can be their own sections within the main content, but avoid generic or vague headings like “Loading Data” and instead use descriptive headings pertinent to the content of the tutorial and the actual data being downloaded or files being used.

>If the user needs to download data to run the tutorial properly, where possible, use [Astroquery](https://astroquery.readthedocs.io/en/latest/) (or similar) to retrieve files. If this is not possible, see the [data guide](https://github.com/spacetelescope/style-guides/blob/master/guides/where-to-put-your-data.md) for other options.

If we didn't know the the specific 'key' or unique string that identifies the dataset we want, we could query Vizier by the author name. In the first query below, ```catalog_list1``` is an ordered dictionary. Alternatively, we could search for any catalogs that could be found with the key words 'APOGEE' and 'binary,' as we do for catalog_list2.

In [2]:
catalog_list1 = Vizier.find_catalogs('Price-Whelan') 
print(str(len(catalog_list1.items()))+' results for "Price-Whelan" query:')
print({k:v.description for k,v in catalog_list1.items()})
catalog_list2 = Vizier.find_catalogs('APOGEE binary') 
print(str(len(catalog_list2.items()))+' results for "APOGEE binary" query:')
print({k:v.description for k,v in catalog_list2.items()})

17 results for "Price-Whelan" query:
{'J/ApJ/760/12': 'LIGO/Virgo gravitational-wave (GW) bursts with GRBs (Abadie+, 2012)', 'J/ApJ/785/119': 'Gravitational waves from known pulsars (Aasi+, 2014)', 'J/ApJ/809/59': 'Ophiuchus stellar stream with PS1 data (Sesar+, 2015)', 'J/ApJ/813/39': 'LIGO gravitational-wave (GW) searches from SNRs (Aasi+, 2015)', 'J/ApJ/816/L4': 'Candidate BHB stars in Ophiuchus stream (Sesar+, 2016)', 'J/ApJ/838/107': 'Distances to RRab stars from WISE and Gaia (Sesar+, 2017)', 'J/ApJ/854/47': 'RRab stars of Monoceros Ring & A13 overdensities (Sheffield+, 2018)', 'J/ApJ/859/L8': 'Properties of TriAnd stars (Hayes+, 2018)', 'J/ApJ/866/133': 'Continuum-H{beta} light curves of 5 Seyfert 1 (De Rosa+, 2018)', 'J/ApJ/887/19': 'DECam phot. of Gaia stars in Price-Whelan 1 (Price-Whelan+, 2019)', 'J/ApJ/887/115': 'Spectra of 28 stars in Price-Whelan 1 association (Nidever+, 2019)', 'J/ApJ/889/63': 'Properties of Sgr Stars (Hayes+, 2020)', 'J/AJ/153/257': 'Comoving stars in 

Using either of the results above, we can find that 'J/AJ/156/18' is the key that matches the paper we were looking for. 

It's also possible to skip the searching step. When working with data from a publication, look for the "Data Products" or "Related Materials" sections of its entry in [ADS](https://ui.adsabs.harvard.edu/) for direct links to online supplemental material. In our case, "Catalog: 2019yCat..51560018P" is listed under related materials; this is the ADS entry for the catalog itself. "CDS(1)" is listed under "Data Products"; this is the link to the VizieR entry, where we can see that the catalog key is J/AJ/156/18. With this key, we can directly load all the tables in this catalog. 

In [3]:
catalogs=Vizier.get_catalogs('J/AJ/156/18')
print(catalogs)

TableList with 8 tables:
	'0:J/AJ/156/18/table2' with 5 column(s) and 50 row(s) 
	'1:J/AJ/156/18/table3' with 1 column(s) and 50 row(s) 
	'2:J/AJ/156/18/table4a' with 13 column(s) and 50 row(s) 
	'3:J/AJ/156/18/table4b' with 10 column(s) and 50 row(s) 
	'4:J/AJ/156/18/table4c' with 12 column(s) and 50 row(s) 
	'5:J/AJ/156/18/table5a' with 12 column(s) and 50 row(s) 
	'6:J/AJ/156/18/table5b' with 6 column(s) and 50 row(s) 
	'7:J/AJ/156/18/table5c' with 6 column(s) and 50 row(s) 


There are actually multiple tables available within this catalog entry, so we'll need to investigate more to find which ones contain the list of binary star systems. Furthermore, the title of the paper says there should be about 5000 sources in the catalog, so the "50 row(s)" shown above must mean our tables are being truncated. Using the command below, we'll remove the row limit and retrieve the catalog again. 

In [13]:
Vizier.ROW_LIMIT = -1
catalogs=Vizier.get_catalogs('J/AJ/156/18')
print(catalogs)

TableList with 8 tables:
	'0:J/AJ/156/18/table2' with 5 column(s) and 96231 row(s) 
	'1:J/AJ/156/18/table3' with 1 column(s) and 4898 row(s) 
	'2:J/AJ/156/18/table4a' with 13 column(s) and 320 row(s) 
	'3:J/AJ/156/18/table4b' with 10 column(s) and 320 row(s) 
	'4:J/AJ/156/18/table4c' with 12 column(s) and 320 row(s) 
	'5:J/AJ/156/18/table5a' with 12 column(s) and 212 row(s) 
	'6:J/AJ/156/18/table5b' with 6 column(s) and 212 row(s) 
	'7:J/AJ/156/18/table5c' with 6 column(s) and 212 row(s) 


The tables in the TableList can be referenced by their integer number (```catalogs[0]```) or named key (```catalogs['J/AJ/156/18/table2']```). By the length alone, ```catalogs[1]``` appears to be the list of the nearly 5000 binary candidates. Reading the source paper itself lets us know exactly what these tables show:

- Table 2 contains the 96,231 stars that were the parent sample for this work.
- Table 3 contains the 4898 stars that likely have a companion, but orbital properties could not be constrained.
- Tables 4a, 4b, and 4c contain the 320 systems with uniquely determined companion orbits.
- Tables 5a, 5b, and 5c contain the 106 systems with two distinct companion orbit possibilities each.

While any of these stars may have visible eclipses observed by TESS, we'll start out search with the most likely candidates. The larger the mass of the evolved star's companion, the brigher it will be, and the change in the observed luminosity of the stellar system will be the most pronounced during an eclipse. Furthermore, a system with a shorter orbital period will also be the most likely to have an eclipse successfully observed by TESS. Let's focus on one of the stellar systems with already determined orbits (Table 4) to demonstrate how to find its corresponding lightcurve from TESS. 

#### Selecting one stellar system from the catalog's Table 4

We need to see which columns of data are in which table, since Table 4 is separated into 3 parts. 

In [30]:
catids=[2,3,4]
for i in catids:
    print(catalogs[i].columns)

<TableColumns names=('APOGEE','M0','omega','K','Teff','logg','__Fe_H_','logM','Jmag','Hmag','Ksmag','RAJ2000','DEJ2000')>
<TableColumns names=('APOGEE','TeffA','loggA','Vmicro','Vmacro','vsini','__Z_H_','__a_M_','chi2A','TClass')>
<TableColumns names=('APOGEE','__C_Fe_','__CI_Fe_','__N_Fe_','__O_Fe_','__Na_Fe_','__Mg_Fe_','_4.5magW','_4.5targ','EK','pmRA','pmDE')>


We wanted to make our selections based on the period and the relative masses of the two stars in the system, but those columsn are not included here! Not all columns are selected by default with VizieR. Visiting [the VizieR site itself for Table 4a](https://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=J/AJ/156/18/table4a) shows us which columns are included by default as well as the names of the other columns we are missing. Let's redo our call to VizieR to specifically choose the columns we want. We can do this by creating a new instance of the ```VizierClass``` that will only include the columns we choose. From [Table 4a's online version in VizieR](https://vizier.cfa.harvard.edu/viz-bin/VizieR-3?-source=J/AJ/156/18/table4a), we can find the keys for the following columns:

- APOGEE, the identifier used by APOGEE.
- Per, the period in days.
- M1, the primary mass estimate.
- M2min, the minimum companion mass.
- qmin, the minimum mass ratio.
- RAJ2000, Right Ascension in decimal degrees (J2000)
- DECJ2000, Declination in decimal degrees (J2000)

Additionally, we can also filter the rows to only include those stellar systems that have a qmin, or minimum mass ratio of the companion to the primary star, of 0.3 and above.

Finally, we'll rename Table 4a as tbl and sort it in ascending order by the period in days.

In [43]:
v = Vizier(columns=['APOGEE','Per','M1','M2min','qmin','RAJ2000', 'DEJ2000'],
           column_filters={"qmin":">0.3"})
v.ROW_LIMIT = -1
catalogs=v.get_catalogs('J/AJ/156/18')
tbl=catalogs[2]
tbl.sort('Per')
print(tbl)

      APOGEE          Per        M1     M2min     qmin    RAJ2000    DEJ2000  
                       d      solMass  solMass              deg        deg    
------------------ ---------- -------- -------- -------- ---------- ----------
2M00092789+0145417    2.61133  0.78317  0.40848  0.52157   2.366216   1.761607
2M00104203+0152065   23.47142  0.92806  0.36483  0.39311   2.675149   1.868474
2M19364967+3813244   24.72355  0.88201  0.31293  0.35479 294.206988  38.223454
2M20183197+1953430   51.30696  1.96604  1.36037  0.69193 304.633241  19.895283
2M01210284+8431304   54.25957  2.25258  0.96650  0.42906  20.261862  84.525131
2M04411627+5855354   56.05161  1.46516  0.61969  0.42295  70.317793  58.926521
2M07103169+0712585   62.37094  1.65363  0.58727  0.35514 107.632065   7.216252
2M18554795-0751284   72.52728  0.59656  0.42263  0.70845 283.949829  -7.857901
2M07282763+2225408   80.29508  1.91156  0.97309  0.50906 112.115125  22.428013
2M00085727+7341257   83.66193  1.77608  1.12293  0.6

Our table has been narrowed down from 320 stellar systems to only 39 that meet our criteria. We'll use the entry at the top, ```2M00092789+0145417```, for the remainder of the tutorial.

## Exercise - how to format these when embedded throughout
Exercise 1: Instead of filtering by ```qmin``` and sorting by period, filter by period in days and then sort by ```qmin```. How do your resulting tables vary? Which one do you think would be most helpful in trying to find eclipses, and why? 

In [45]:
# Place for code for Exercise 1

# Searching the TASOC archive for a lightcurve by coordinate

It's possible to search the MAST archive by object name, but the APOGEE names are not resolvable into a sky position. Instead, we will search by the RA and Dec coordinates; the default radius is 0.2 degrees.

In [69]:
ind=0
coordstring=str(tbl['RAJ2000'].data[ind])+' '+str(tbl['DEJ2000'].data[ind])
print(coordstring)
obs_tbl = Observations.query_region(coordstring)
print(obs_tbl)

2.366216 1.761607
intentType obs_collection provenance_name instrument_name ... srcDen  obsid        distance     
---------- -------------- --------------- --------------- ... ------ -------- ------------------
   science           TESS            SPOC      Photometer ...    nan 63630590  239.9102423920586
   science           TESS            SPOC      Photometer ...    nan 63630591  406.8753727790844
   science    SPITZER_SHA    SSC Pipeline            IRAC ...    nan  1713755  628.0880608444758
   science    SPITZER_SHA    SSC Pipeline            IRAC ...    nan  1713755  628.0880608444758
   science            PS1             3PI            GPC1 ... 5885.0  2102451                0.0
   science            PS1             3PI            GPC1 ... 5885.0  2102452                0.0
   science            PS1             3PI            GPC1 ... 5885.0  2102453                0.0
   science            PS1             3PI            GPC1 ... 5885.0  2102454                0.0
   science  

The MAST archive has 42 observations of the stellar system. However, we don't want to access the raw observations; we are interested in observations that have already been processed and transformed into a lightcurve; the lightcurve itself is what is called a "High Level Science Product" or HLSP. The products we want will have "HLSP" in the ```obs_collection``` column and "TASOC" in the ```provenance_name``` column.

### File Information

Explain pertinent details about the file you've just downloaded. For example, if working with Kepler light curves, explain what's in the different file extensions:

```
- No. 0 (Primary): This HDU contains metadata related to the entire file.
- No. 1 (Light curve): This HDU contains a binary table that holds data like
  flux measurements and times. We will extract information from here when we
  define the parameters for the light curve plot.
- No. 2 (Aperture): This HDU contains the image extension with data collected
  from the aperture. We will also use this to display a bitmask plot that
  visually represents the optimal aperture used to create the SAP_FLUX column in
  HDU1.

```

Where possible (if the code supports it), use code examples that visually display the data in the tutorial. For example, if you are showing an object such as a Table, display a preview of the table:

In [15]:
yourProd[0:5]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid
str10,str6,str10,str36,str59,str1,str110,str7,str28,str1,str1,str6,str1,str7,str44,int64,str10
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Preview-Full,C,mast:Kepler/url/missions/kepler/previews/0089/008957091/kplr008957091-2013098041711_llc_bw_large.png,PREVIEW,--,--,--,Kepler,--,GO30032,kplr008957091-2013098041711_llc_bw_large.png,29653,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q10,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2011271113734_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2011271113734_llc.fits,486720,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q11,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2012004120508_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2012004120508_llc.fits,506880,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q12,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2012088054726_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2012088054726_llc.fits,434880,9000159893
9000159893,Kepler,timeseries,kplr008957091_lc_Q000000000011111111,Lightcurve Long Cadence (CLC) - Q13,C,mast:Kepler/url/missions/kepler/lightcurves/0089/008957091/kplr008957091-2012179063303_llc.fits,SCIENCE,--,--,--,Kepler,--,GO30032,kplr008957091-2012179063303_llc.fits,472320,9000159893


In [ ]:
# Download the products
Observations.download_products(yourProd, mrp_only=False, cache=False)

## Exercises
Exercises are optional, but encouraged. Exercises can be woven into the main content of your tutorial, or appear in their own section toward the end of the tutorial. Final exercises can be more challenging, similar to homework problems. They can be minimal or take as long as 30 minutes to an hour to complete. If you do have one or more exercises in your tutorial, be sure to leave a blank code cell underneath each to show the reader that they're meant to try out their new skill right there. You may also want to include a "solutions" notebook next to your main notebook for the reader to check their work after they have finished their attempt.

## Additional Resources

>This section is optional. Try to weave resource links into the main content of your tutorial so that they are falling in line with the context of your writing. For resources that do not fit cleanly into your narrative, you may include an additional resources section at the end of your tutorial. Usually a list of links using Markdown bullet list plus link format is appropriate:

- [Astroquery:docs, VizieR Queries](https://astroquery.readthedocs.io/en/latest/vizier/vizier.html)

## About this Notebook
>Let the world know who the author of this great tutorial is! If possible and appropriate, include a contact email address for users who might need support (for example, `archive@stsci.edu`). You can also optionally include keywords, your funding source, or a last update date in this section.

**Author:** Julia Kamenetzky, ScienceBetter Consultant
**Updated On:** YYYY-MM-DD

## Citations
>Provide your reader with guidelines on how to cite open source software and other resources in their own published work.

If you use `astropy` or `lightkurve` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)
* [Citing `VizieR`](https://cds.unistra.fr/vizier-org/licences_vizier.html?#copyrightvizier)

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 